
# Wrapt Library Deep Dive

## Table of Contents
1. [Standard Decorator Approach](#Standard-Decorator-Approach)
2. [Decorator Factory Pattern](#Decorator-Factory-Standard-Approach)
3. [Wrapt Optimizations](#Using-Wrapt)
4. [Practical Use Cases](#Use-Case)
5. [Complatible with all callables](#Compatibility)

In [1]:
!pip install wrapt
!pip install dateutils

In [1]:
import logging
import inspect  # Used for the introspection section
from functools import wraps

import wrapt
from dateutil import parser

In [2]:
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(fmt="{asctime} - {levelname} - {message}", datefmt="%Y-%m-%d %H:%M:%S", style="{")
# Add formatter to Stream Handler
handler.setFormatter(formatter)
# Add handler to the logger
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)
logger.debug("This is debug message")

2025-06-22 20:29:36 - DEBUG - This is debug message


### Standard Decorator Approach

In [3]:
def log(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        try:
            result = fn(*args, **kwargs)
            logger.info(f"called {fn.__name__} with {args=}, {kwargs=}")
            return result
        except Exception as ex:
            logger.error(f"called {fn.__name__} with {args=}, {kwargs=}, {ex=}")
            raise

    return wrapper


@log
def add(a, b):
    """A fucntion that adds two numbers"""
    return a + b


add(1, 2)

2025-06-22 20:29:40 - INFO - called add with args=(1, 2), kwargs={}


3

In [4]:
try:
    add("a", 1)
except Exception as ex:
    print("Exception was raised:", type(ex), str(ex))

2025-06-22 20:29:41 - ERROR - called add with args=('a', 1), kwargs={}, ex=TypeError('can only concatenate str (not "int") to str')


Exception was raised: <class 'TypeError'> can only concatenate str (not "int") to str


### Decorator Factory Standard Approach


Let's create a decorator that can take a keyword only argument to silence or propagate the error if exists. Moreover we want to support the typical decorator syntax if no other argument passed when we apply the decorator.

In [5]:
def log(_func=None, *, propagate_exception=True):
    def decorator(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            try:
                result = fn(*args, **kwargs)
                logger.info(f"called {fn.__name__} with {args=}, {kwargs=}")
                return result
            except Exception as ex:
                logger.error(f"called {fn.__name__} with {args=}, {kwargs=}, {ex=}")
                if propagate_exception:
                    raise

        return wrapper

    if callable(_func):
        return decorator(_func)

    return decorator


# Without parentheses
@log
def add(a, b):
    """A fucntion that adds two numbers"""
    return a + b


add(1, 2)


# With parentheses and parameter
@log(propagate_exception=False)
def add(a, b):
    """A fucntion that adds two numbers"""
    return a + b


add(1, "a")


# With parentheses and parameter
@log(propagate_exception=True)
def add(a, b):
    """A fucntion that adds two numbers"""
    return a + b


try:
    add(1, "a")
except Exception as ex:
    print(ex)


# With default parameters (parentheses but no args)
@log()
def function3():
    pass


add(1, "a")

2025-06-22 20:29:44 - INFO - called add with args=(1, 2), kwargs={}
2025-06-22 20:29:44 - ERROR - called add with args=(1, 'a'), kwargs={}, ex=TypeError("unsupported operand type(s) for +: 'int' and 'str'")
2025-06-22 20:29:44 - ERROR - called add with args=(1, 'a'), kwargs={}, ex=TypeError("unsupported operand type(s) for +: 'int' and 'str'")
2025-06-22 20:29:44 - ERROR - called add with args=(1, 'a'), kwargs={}, ex=TypeError("unsupported operand type(s) for +: 'int' and 'str'")


unsupported operand type(s) for +: 'int' and 'str'


TypeError: unsupported operand type(s) for +: 'int' and 'str'

### Using Wrapt

wrapts.decorator essentially helps optimize decorator implementation by reducing closure nesting.

Note :
   -  A simple decorator has two nested functions

In [6]:
# wrapt.decorator takes a decorator and return another decorator which has access to the wrapped function __name__, __module__, __doc__
# it also has access to the argument and keyword argument to the wrapped function
# Correctly forward those arguments on the call of the wrapped function


@wrapt.decorator
def log(wrapped, instance, args, kwargs):
    try:
        result = wrapped(*args, **kwargs)
        logger.info(f"called {wrapped.__name__} with {args=}, {kwargs=}")
        return result
    except Exception as ex:
        logger.error(f"called {wrapped.__name__} with {args=}, {kwargs=}, {ex=}")
        raise


@log
def add(a, b):
    """A fucntion that adds two numbers"""
    return a + b


add(1, 2)

2025-06-22 20:29:49 - INFO - called add with args=(1, 2), kwargs={}


3

### Using Wrapt for a Decorator Factory

In [7]:
def log(_func=None, *, propagate_exception=True):

    @wrapt.decorator  # Decorator
    def wrapper(wrapped, instance, args, kwargs):
        try:
            result = wrapped(*args, **kwargs)
            logger.info(f"called {wrapped.__name__} with {args=}, {kwargs=}")
            return result
        except Exception as ex:
            logger.error(f"called {wrapped.__name__} with {args=}, {kwargs=}, {ex=}")
            if propagate_exception:
                raise

    if callable(_func):
        return wrapper(_func)

    return wrapper

In [8]:
# Without parentheses
@log
def add(a, b):
    """A fucntion that adds two numbers"""
    return a + b


add(1, 2)


# With parentheses and parameter
@log(propagate_exception=False)
def add(a, b):
    """A fucntion that adds two numbers"""
    return a + b


add(1, "a")


# With parentheses and parameter
@log(propagate_exception=True)
def add(a, b):
    """A fucntion that adds two numbers"""
    return a + b


try:
    add(1, "a")
except Exception as ex:
    print(ex)


# With parentheses and parameter
@log()
def add(a, b):
    """A fucntion that adds two numbers"""
    return a + b


add(1, "a")

2025-06-22 20:29:54 - INFO - called add with args=(1, 2), kwargs={}
2025-06-22 20:29:54 - ERROR - called add with args=(1, 'a'), kwargs={}, ex=TypeError("unsupported operand type(s) for +: 'int' and 'str'")
2025-06-22 20:29:54 - ERROR - called add with args=(1, 'a'), kwargs={}, ex=TypeError("unsupported operand type(s) for +: 'int' and 'str'")
2025-06-22 20:29:54 - ERROR - called add with args=(1, 'a'), kwargs={}, ex=TypeError("unsupported operand type(s) for +: 'int' and 'str'")


unsupported operand type(s) for +: 'int' and 'str'


TypeError: unsupported operand type(s) for +: 'int' and 'str'

### Let's say we want to turn off, and turn on the decorators using a configuration.

Wrapt library provides a built-in functionality to do it but this configurations is specified once the program starts and cannot change on runtime.

In [9]:
ENABLE_LOG = True


def log(_func=None, *, propagate_exception=True):

    @wrapt.decorator(enabled=ENABLE_LOG)
    def wrapper(wrapped, instance, args, kwargs):
        try:
            result = wrapped(*args, **kwargs)
            logger.info(f"called {wrapped.__name__} with {args=}, {kwargs=}")
            return result
        except Exception as ex:
            logger.error(f"called {wrapped.__name__} with {args=}, {kwargs=}, {ex=}")
            if propagate_exception:
                raise

    if callable(_func):
        return wrapper(_func)

    return wrapper


@log
def add(a, b):
    """A fucntion that adds two numbers"""
    return a + b


add(1, 2)

2025-06-22 20:29:58 - INFO - called add with args=(1, 2), kwargs={}


3

In [10]:
# The LOG_ENABLED was passed when the decorator wraps the initial function, so even if I change the configuration now it will not be applied dynamically to the decorator.
ENABLE_LOG = False
add(1, 2)

2025-06-22 20:30:00 - INFO - called add with args=(1, 2), kwargs={}


3

If this global configuration approach is not enough wrapt provides a hook to turn off or turn on the decorators on runtime. Instead of passing a boolean value, pass a predicate function that turn on or turn the decorator.

In [11]:
ENABLE_LOG = True


def activate_logs():
    global ENABLE_LOG
    return ENABLE_LOG


def log(_func=None, *, propagate_exception=True):

    @wrapt.decorator(enabled=activate_logs)
    def wrapper(wrapped, instance, args, kwargs):
        try:
            result = wrapped(*args, **kwargs)
            logger.info(f"called {wrapped.__name__} with {args=}, {kwargs=}")
            return result
        except Exception as ex:
            logger.error(f"called {wrapped.__name__} with {args=}, {kwargs=}, {ex=}")
            if propagate_exception:
                raise

    if callable(_func):
        return wrapper(_func)

    return wrapper


@log
def add(a, b):
    """A fucntion that adds two numbers"""
    return a + b


add(1, 2)

2025-06-22 20:30:03 - INFO - called add with args=(1, 2), kwargs={}


3

In [12]:
# No logging.
ENABLE_LOG = False
add(1, 2)

3

### Introspection PlainDecorator vs Wraps vs Wrapt

In [13]:
def dec(fn):
    def inner(*args, **kwargs):
        return fn(*args, **kwargs)

    return inner


def wraps_dec(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        return dec(fn(*args, **kwargs))

    return inner


@wrapt.decorator
def wrapt_dec(wrapped, instance, args, kwargs):
    return wrapped(*args, **kwargs)


@dec
def add(a, b, *, extras=None):
    """
    A function that adds two numbers.
        a: first number
        b: second number
        extras: extra arguments just for demonstration purposes
    """
    return a + b


@wraps_dec
def add_wraps(a, b, *, extras=None):
    """
    A function that adds two numbers.
        a: first number
        b: second number
        extras: extra arguments just for demonstration purposes
    """
    return a + b


@wrapt_dec
def add_wrapt(a, b, *, extras=None):
    """
    A function that adds two numbers.
        a: first number
        b: second number
        extras: extra arguments just for demonstration purposes
    """
    return a + b

In [14]:
print(inspect.getdoc(add), inspect.getdoc(add_wraps), inspect.getdoc(add_wrapt), sep="\n-----------\n")

None
-----------
A function that adds two numbers.
    a: first number
    b: second number
    extras: extra arguments just for demonstration purposes
-----------
A function that adds two numbers.
    a: first number
    b: second number
    extras: extra arguments just for demonstration purposes


In [15]:
print(
    inspect.getfullargspec(add),
    inspect.getfullargspec(add_wraps),
    inspect.getfullargspec(add_wrapt),
    sep="\n-----------\n",
)

FullArgSpec(args=[], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})
-----------
FullArgSpec(args=[], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})
-----------
FullArgSpec(args=['a', 'b'], varargs=None, varkw=None, defaults=None, kwonlyargs=['extras'], kwonlydefaults={'extras': None}, annotations={})


Notice that the plain wraps decorator grab the arguments from the inner function and not from the decorated function.

On the other hand, wrapt does better to this matter. It actually grabs the arguments, keyword arguments of the decorated function.

## Use Case

Assuming that we want to use the wrapt decorator to apply the Open/Closed principal meaning no modification to the core logic of the wrapped function but we can extend what it does.
For example in data engineering, it is required to validate the input of a transformation function or the output of this function quite often.

Let's say the applied transformation take place in a record represented as dictionary or in as row of a pd.Dataframe.

What we want is to execute a set of instruction(function) before or after the transformation logic. This can be achieved via decorators with the steps below:

1. Turn the additional set of instruction(function) to a decorator
2. Apply this decorator the the tranformation logic

Cases:

    - the set of instruction applied before the transformation(validation, inputs_preprocessing)
    - the set of instruction applied after the transformation(post-process output, validate the validity of the output)
    - Since decorators can be stacked it is prossible to create a pipeline

In [16]:
records_list = [
    {"id": 1, "temperature": 98.6, "status": "healthy", "timestamp": "2023-01-01T12:00:00"},
    {"id": 2, "temperature": 101.2, "status": "fever", "timestamp": None},
    {"id": 3, "temperature": None, "status": "unknown", "timestamp": "2023-01-01T12:10:00"},
]


def fahrenheit_to_celsius(record):
    """Core logic: Convert temperature from °F to °C (no validation)."""
    if record["temperature"] is not None:
        record["temperature"] = (record["temperature"] - 32) * 5 / 9
    return record


def cast_datetime(record):
    """Core logic: Convert datetime strings to datetime objects (no validation)."""
    record["timestamp"] = parser.parse(record["timestamp"])
    return record

In [17]:
# Without validation for None in timestamp we will end up in error
# Moreover we want to validate that the temperature is above ten celcius degrees
for row in records_list:
    try:
        print(cast_datetime(fahrenheit_to_celsius(row)))
    except TypeError as ex:
        print(ex)
        row["timestamp"] = "1990-01-01T00:00:00"  # default value to identify None afterwards
        print(cast_datetime(row))

{'id': 1, 'temperature': 37.0, 'status': 'healthy', 'timestamp': datetime.datetime(2023, 1, 1, 12, 0)}
Parser must be a string or character stream, not NoneType
{'id': 2, 'temperature': 38.44444444444444, 'status': 'fever', 'timestamp': datetime.datetime(1990, 1, 1, 0, 0)}
{'id': 3, 'temperature': None, 'status': 'unknown', 'timestamp': datetime.datetime(2023, 1, 1, 12, 10)}


Let's replace the try and except block using an approach look-before leap implemented from a decorator.

In [18]:
@wrapt.decorator
def validate_timestamp(wrapped, instance, args, kwargs):
    def _execute(record, *args, **kwargs):
        print(f"{args=}, {kwargs=}")
        if record["timestamp"] is None:
            record["timestamp"] = "1990-01-01T00:00:00"

        return wrapped(record, *args, **kwargs)

    return _execute(*args, **kwargs)


@validate_timestamp
def cast_datetime(record, other_arg, *, extras="only for demonstration purposes"):
    """Core logic: Convert datetime strings to datetime objects (no validation)."""
    record["timestamp"] = parser.parse(record["timestamp"])
    return record


records_list = [
    {"id": 1, "temperature": 98.6, "status": "healthy", "timestamp": "2023-01-01T12:00:00"},
    {"id": 2, "temperature": 101.2, "status": "fever", "timestamp": None},
    {"id": 3, "temperature": None, "status": "unknown", "timestamp": "2023-01-01T12:10:00"},
]

for row in records_list:
    print(
        fahrenheit_to_celsius(
            cast_datetime(
                row, "demonstration_purposes", extras="only for demonstration purposes"
            )  # Return only the record
        )
    )

args=('demonstration_purposes',), kwargs={'extras': 'only for demonstration purposes'}
{'id': 1, 'temperature': 37.0, 'status': 'healthy', 'timestamp': datetime.datetime(2023, 1, 1, 12, 0)}
args=('demonstration_purposes',), kwargs={'extras': 'only for demonstration purposes'}
{'id': 2, 'temperature': 38.44444444444444, 'status': 'fever', 'timestamp': datetime.datetime(1990, 1, 1, 0, 0)}
args=('demonstration_purposes',), kwargs={'extras': 'only for demonstration purposes'}
{'id': 3, 'temperature': None, 'status': 'unknown', 'timestamp': datetime.datetime(2023, 1, 1, 12, 10)}


It is ok to set an arbitrary value to timestamp if missing but we do not want to handle with grace invalid temperatures like None.

In [19]:
@wrapt.decorator
def validate_temperature(wrapped, instance, args, kwargs):
    def _execute(record, *args, **kwargs):
        result = wrapped(record, *args, **kwargs)

        # Applied logic after the decorated function has returned
        if result["temperature"] is None:
            raise ValueError("Invalid temperature: {}".format(result["temperature"]))
        return result

    return _execute(*args, **kwargs)


@wrapt.decorator
def fill_timestamp(wrapped, instance, args, kwargs):
    def _execute(record, *args, **kwargs):
        # Logic before the execution of wrapped function
        if record["timestamp"] is None:
            record["timestamp"] = "1990-01-01T00:00:00"

        return wrapped(record, *args, **kwargs)

    return _execute(*args, **kwargs)


@validate_temperature
def fahrenheit_to_celsius(record):
    """Core logic: Convert temperature from °F to °C (no validation)."""
    if record["temperature"] is not None:
        record["temperature"] = (record["temperature"] - 32) * 5 / 9
    return record


@fill_timestamp
def cast_datetime(record):
    """Core logic: Convert datetime strings to datetime objects (no validation)."""
    record["timestamp"] = parser.parse(record["timestamp"])
    return record

It will raise our custom value error in the third record

In [20]:
records_list = [
    {"id": 1, "temperature": 98.6, "status": "healthy", "timestamp": "2023-01-01T12:00:00"},
    {"id": 2, "temperature": 101.2, "status": "fever", "timestamp": None},
    {"id": 3, "temperature": None, "status": "unknown", "timestamp": "2023-01-01T12:10:00"},
]

for row in records_list:
    print(fahrenheit_to_celsius(cast_datetime(row)))

{'id': 1, 'temperature': 37.0, 'status': 'healthy', 'timestamp': datetime.datetime(2023, 1, 1, 12, 0)}
{'id': 2, 'temperature': 38.44444444444444, 'status': 'fever', 'timestamp': datetime.datetime(1990, 1, 1, 0, 0)}


ValueError: Invalid temperature: None

### Compatibility

In [21]:
@wrapt.decorator
def printer(wrapped, instance, args, kwargs):
    print(f"{wrapped=}, {instance=}, {args=}, {kwargs=}", end="\n\n")
    return wrapped(*args, **kwargs)


@printer
def my_function(arg1, arg2, arg3):
    return ", ".join(map(str, [arg1, arg2, arg3]))


print(my_function(1, 2, arg3=3), end="\n\n------------------------\n")


class Class(object):

    @printer
    def __init__(self, a, b, *, c):
        self.a = a
        self.b = b
        self.c = c

    @printer
    @classmethod
    def cls_function(cls, arg1, arg2, arg3):
        return "| ".join(map(str, [arg1, arg2, arg3]))

    @printer
    @staticmethod
    def static_function(arg1, arg2, arg3):
        return "~ ".join(map(str, [arg1, arg2, arg3]))


obj = Class(1, 2, c=3)
print(f"id(obj): {hex(id(obj))}")
print(obj.cls_function("a", "b", arg3="c"))
print(obj.static_function("a", "b", arg3="c"))

wrapped=<function my_function at 0x000002783F9E9760>, instance=None, args=(1, 2), kwargs={'arg3': 3}

1, 2, 3

------------------------
wrapped=<bound method Class.__init__ of <__main__.Class object at 0x000002783F29BB60>>, instance=<__main__.Class object at 0x000002783F29BB60>, args=(1, 2), kwargs={'c': 3}

id(obj): 0x2783f29bb60
wrapped=<bound method Class.cls_function of <class '__main__.Class'>>, instance=<class '__main__.Class'>, args=('a', 'b'), kwargs={'arg3': 'c'}

a| b| c
wrapped=<function Class.static_function at 0x000002783F9EA3E0>, instance=None, args=('a', 'b'), kwargs={'arg3': 'c'}

a~ b~ c


#### Decorate classes


Let's say we want to decorate all the callables with our printer decorator within a class.

###### Reminder !!!
When the wrapt.decorator is applied to a function, the instance is None, when applied to an instance method then the instance=self
Let's create a decorator that add the printer decorator to all the callables of a Class.

In [22]:
@wrapt.decorator
def debug(wrapped, instance, args, kwargs):
    for attr, value in vars(wrapped).items():
        if callable(value):
            setattr(wrapped, attr, printer(value))
    return wrapped(*args, **kwargs)  # Return a new class with all the callables decorated


@debug
class DecClass(object):

    def __init__(self, a, b, *, c):
        self.a = a
        self.b = b
        self.c = c

    @classmethod
    def cls_function(cls, arg1, arg2, arg3):
        return "| ".join(map(str, [arg1, arg2, arg3]))

    @staticmethod
    def static_function(arg1, arg2, arg3):
        return "~ ".join(map(str, [arg1, arg2, arg3]))

In [23]:
obj = DecClass("a", "b", c="c")  # Notice that it is executed when the __init__ was called. Not when the __new__

wrapped=<bound method DecClass.__init__ of <__main__.DecClass object at 0x000002783F29BCB0>>, instance=<__main__.DecClass object at 0x000002783F29BCB0>, args=('a', 'b'), kwargs={'c': 'c'}



In [24]:
# There is not __new__ in the dictionary of the class since it is inherited from the object class
vars(Class)

mappingproxy({'__module__': '__main__',
              '__firstlineno__': 15,
              '__init__': <function __main__.Class.__init__(self, a, b, *, c)>,
              'cls_function': <FunctionWrapper at 0x000002783F9D5460 for classmethod at 0x000002783F462290>,
              'static_function': <FunctionWrapper at 0x000002783F9D53F0 for staticmethod at 0x000002783F463520>,
              '__static_attributes__': ('a', 'b', 'c'),
              '__dict__': <attribute '__dict__' of 'Class' objects>,
              '__weakref__': <attribute '__weakref__' of 'Class' objects>,
              '__doc__': None})

In [25]:
obj.cls_function(1, 2, 3)

'1| 2| 3'

In [26]:
obj.static_function(1, 2, 3)

wrapped=<function DecClass.static_function at 0x000002783F9EA8E0>, instance=None, args=(1, 2, 3), kwargs={}



'1~ 2~ 3'